This file filters our cdp data to remove all companies that have not applicable or empty values

Currently works for 2022 4.1a data

we need to process 4.1a and 4.1b and see whether companies answer one, the other, or both

In [2]:
import pandas as pd
import csv

#Read the CSV file
input_file = 'raw_files_4.1b/2018_c4.1b.csv'
df = pd.read_csv(input_file)


#question = "C4.1a_C16_Provide details of your absolute emissions target(s) and progress made against those targets. - Please explain (including target coverage)"
question = "C4.1b_C13_Provide details of your emissions intensity target(s) and progress made against those target(s). - Please explain"


In [3]:

#Drop rows with empty values in the '4.1a_C29' column
df = df.dropna(subset=[question])

# Step 2: Filter out rows with "not applicable" in the question column
df = df[df[question] != 'Question not applicable']


KeyError: ['C4.1a_C12_Provide details of your absolute emissions target(s) and progress made against those targets. - Please explain']

In [75]:

# Step 3: Create a new data fram that combine rows that are the same company
fields = ["Organization", "Country", "Primary activity", "Primary sector", "Primary industry", question]


result_file = 'compiled_4.1b_files/Compiled_2018_c4.1b.csv'

with open(result_file, 'w', newline='') as csvfile:
    # creating a csv dict writer object
    writer = csv.DictWriter(csvfile, fieldnames=fields, quoting=csv.QUOTE_ALL)
    writer.writeheader()
    company_name_list = df.loc[:,"Organization"]
    country_list = df.loc[:,"Country"]
    activity_list = df.loc[:,"Primary activity"]
    sector_list = df.loc[:,"Primary sector"]
    industry_list = df.loc[:,"Primary industry"]
    response_list = df.loc[:,question]

    # print("LEN: ", len(df))
    # print("company name length", type(company_name_list))
    #bprint(company_name_list.iloc[2])

    length = 0
    index = 0
    while(index < len(df)):
        first_response = response_list.iloc[index]
        first_company = company_name_list.iloc[index]
        while (index+1 < len(df)):
            next_company = company_name_list.iloc[index+1]
            if (first_company == next_company):
                response_next = response_list.iloc[index+1]
                first_response += response_next
                index += 1
            else:
                break
        line = {"Organization":company_name_list.iloc[index], "Country": country_list.iloc[index], "Primary activity":activity_list.iloc[index], "Primary sector":sector_list.iloc[index], "Primary industry":industry_list.iloc[index], question:first_response}
        writer.writerow(line)
        index += 1
        length += 1

    csvfile.close()



print(length)
# # Step 3: Save the filtered DataFrame to a new CSV file
# output_file = 'Filtered_2018_c4.1a.csv'
# df.to_csv(output_file, index=False)

print(f"Filtered data has been saved to {result_file}")

TypeError: can only concatenate str (not "float") to str

The following takes in the above filtered data and filters out all non-european countries

In [71]:
#Read the CSV file
input_file = 'Filtered_2018_c4.1b.csv'
df = pd.read_csv(input_file)

european_countries = [
    'Albania', 'Andorra', 'Armenia', 'Austria', 'Azerbaijan', 'Belarus', 'Belgium', 'Bosnia and Herzegovina',
    'Bulgaria', 'Croatia', 'Cyprus', 'Czech Republic', 'Denmark', 'Estonia', 'Finland', 'France', 'Georgia', 'Germany',
    'Greece', 'Hungary', 'Iceland', 'Ireland', 'Italy', 'Kazakhstan', 'Kosovo', 'Latvia', 'Liechtenstein', 'Lithuania',
    'Luxembourg', 'Malta', 'Moldova', 'Monaco', 'Montenegro', 'Netherlands', 'North Macedonia', 'Norway', 'Poland',
    'Portugal', 'Romania', 'Russia', 'San Marino', 'Serbia', 'Slovakia', 'Slovenia', 'Spain', 'Sweden', 'Switzerland',
    'Turkey', 'Ukraine', 'United Kingdom', 'Vatican City'
]

filtered_df = df[df['Country'].isin(european_countries)]

output_file = 'Filtered_European_2018_c4.1b.csv'
filtered_df.to_csv(output_file, index=False)
print(len(filtered_df))
print(f"Filtered data has been saved to {output_file}")

343
Filtered data has been saved to Filtered_European_2018_c4.1b.csv


the folowing code just takes in a file and counts how many companies are in it

In [86]:
import pandas as pd
import csv

#Read the CSV file
input_file = 'raw_files_4.1b/2018_c4.1b.csv'
df = pd.read_csv(input_file)

european_countries = [
    'Albania', 'Andorra', 'Armenia', 'Austria', 'Azerbaijan', 'Belarus', 'Belgium', 'Bosnia and Herzegovina',
    'Bulgaria', 'Croatia', 'Cyprus', 'Czech Republic', 'Denmark', 'Estonia', 'Finland', 'France', 'Georgia', 'Germany',
    'Greece', 'Hungary', 'Iceland', 'Ireland', 'Italy', 'Kazakhstan', 'Kosovo', 'Latvia', 'Liechtenstein', 'Lithuania',
    'Luxembourg', 'Malta', 'Moldova', 'Monaco', 'Montenegro', 'Netherlands', 'North Macedonia', 'Norway', 'Poland',
    'Portugal', 'Romania', 'Russia', 'San Marino', 'Serbia', 'Slovakia', 'Slovenia', 'Spain', 'Sweden', 'Switzerland',
    'Turkey', 'Ukraine', 'United Kingdom', 'Vatican City'
]
filtered_df = df[df['Country'].isin(european_countries)]

company_name_list = filtered_df.loc[:,"Organization"]
index = 0
count = 1

while(index+1 < len(filtered_df)):
    if(company_name_list.iloc[index] != company_name_list.iloc[index+1]):
        count += 1
    index += 1

print("number of european companies: ", count)

number of european companies:  601


the following code takes in the raw files and counts how many different companies are in each sector per year
outputs a csv file with sector and number of companies

now it takes in filtered eurpoean files and counts how many companies are in each industry

In [1]:
import pandas as pd
import csv

#Read the CSV file
#input_file = 'raw_files_4.1b/2022_c4.1b.csv'
# input_file = 'filtered_europe_4.1a_files/Filtered_European_2022_c4.1a.csv'

input_file = 'rating_results_over_years/a_list_all_years_no_reference_with_industry.csv'
df = pd.read_csv(input_file)

# Group by sector and year, then count unique companies in each group
#sector_company_counts = df.groupby(['Primary industry'])['Organization'].nunique().reset_index()
sector_company_counts = df.groupby(['Industry'])['Company'].nunique().reset_index()
sector_company_counts.columns = ['Industry', 'Number of Companies']

# Save the result to a new CSV file
output_file = 'rating_results_over_years/50_non_A_list_industry_company_counts.csv'
sector_company_counts.to_csv(output_file, index=False)


print("CSV file with sector and number of companies has been created:", output_file)


CSV file with sector and number of companies has been created: rating_results_over_years/50_non_A_list_industry_company_counts.csv


In [13]:
import pandas as pd
import csv

#Read the CSV file
#input_file = 'raw_files_4.1b/2022_c4.1b.csv'
input_file = 'filtered_europe_4.1a_files/Filtered_European_2022_c4.1a.csv'
df = pd.read_csv(input_file)

a_list_file = 'a-list_data_files/2022_a_list.csv'
a_df = pd.read_csv(a_list_file)

# Merge the dataframes to filter only the companies in the A list
merged_df = df[df['Organization'].isin(a_df['Company Name'])]

# Group by sector and year, then count unique companies in each group
sector_company_counts = merged_df.groupby(['Primary industry'])['Organization'].nunique().reset_index()

sector_company_counts.columns = ['Industry', 'Number of A List Companies']

# Save the result to a new CSV file
output_file = '2022_A_list_industry_company_counts.csv'
sector_company_counts.to_csv(output_file, index=False)


print("CSV file with sector and number of companies has been created:", output_file)


CSV file with sector and number of companies has been created: 2022_A_list_industry_company_counts.csv


the following code takes in filtered eurpoean files and counts how many are on the A-list in each industry

In [93]:
import pandas as pd
import csv

#Read the CSV file
input_file = 'filtered_europe_4.1a_files/2018_c4.1a.csv'
df = pd.read_csv(input_file)


specified_companies = [
    'FIRMENICH SA', 'L\'Oréal', 'Accenture', 'AIB Group Plc', 'AstraZeneca', 'BASF SE', 'Bayer AG', 
    'Berner Kantonalbank AG BEKB', 'BHP', 'Bic', 'Borregaard ASA', 'Bouygues', 'Brembo SpA', 'BT Group', 
    'Carrefour', 'Coca-Cola European Partners', 'Covivio', 'Danone', 'Deutsche Bahn AG', 'Deutsche Telekom AG', 
    'Diageo Plc', 'DNB ASA', 'EDF', 'Electrolux', 'ENGIE', 'EVRY ASA', 'FERROVIAL', 'Groupe PSA', 
    'Grupo Logista', 'INDUS Holding AG', 'ING Group', 'Intesa Sanpaolo S.p.A', 'J Sainsbury Plc', 'Kering', 
    'Klepierre', 'Koninklijke Philips NV', 'La Poste', 'Landsec', 'Lego Group', 'Lloyds Banking Group', 
    'Mercialys', 'Metsä Board', 'Michelin', 'National Grid PLC', 'Naturgy Energy Group SA', 'Neste Oyj', 
    'Nestlé', 'Nexans', 'Nexity', 'Novo Nordisk A/S', 'Pirelli', 'RELX Group Plc', 'REMA1000', 'Rexel Developpment', 
    'Saint-Gobain', 'Schneider Electric', 'Siemens AG', 'Signify NV', 'Sopra Steria Group', 'Stora Enso Oyj', 
    'Suez', 'Telefonica', 'The NAVIGATOR Company', 'thyssenkrupp AG', 'UBS', 'Unibail-Rodamco-Westfield', 
    'Unilever plc', 'Valeo Sa', 'Valmet', 'Veidekke ASA'
]

filtered_df = df[df['Organization'].isin(specified_companies)]

print(len(filtered_df)-1)

FileNotFoundError: [Errno 2] No such file or directory: 'filtered_europe_4.1a_files/2018_c4.1a.csv'